<a href="https://colab.research.google.com/github/brendanpshea/database_sql/blob/main/data/MarioBrosPlumbing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Connect sql magic
%load_ext sql
%sql sqlite:///mario_bros_plumbing.db

In [2]:
%%sql
CREATE TABLE Customers (
  customer_id INTEGER PRIMARY KEY,
  first_name TEXT,
  last_name TEXT,
  address TEXT,
  phone_number TEXT
);

CREATE TABLE Employees (
  employee_id INTEGER PRIMARY KEY,
  first_name TEXT,
  last_name TEXT,
  job_title TEXT,
  hire_date TEXT
);

CREATE TABLE ServiceTypes (
  service_type_id INTEGER PRIMARY KEY,
  service_type_name TEXT,
  description TEXT
);

CREATE TABLE Services (
  service_id INTEGER PRIMARY KEY,
  service_type_id INTEGER,
  service_name TEXT,
  description TEXT,
  price REAL,
  FOREIGN KEY (service_type_id) REFERENCES ServiceTypes (service_type_id)
);

CREATE TABLE Orders (
  -- Keeps track of a customer's orders
  order_id INTEGER PRIMARY KEY,
  customer_id INTEGER,
  employee_id INTEGER,
  order_date TEXT,
  total_amount REAL,
  FOREIGN KEY (customer_id) REFERENCES Customers (customer_id),
  FOREIGN KEY (employee_id) REFERENCES Employees (employee_id)
);

CREATE TABLE Order_Items (
  -- Keeps track of a customer's order items
  -- This is one line on an invoice
  order_item_id INTEGER PRIMARY KEY,
  order_id INTEGER,
  service_id INTEGER,
  quantity INTEGER,
  FOREIGN KEY (order_id) REFERENCES Orders (order_id),
  FOREIGN KEY (service_id) REFERENCES Services (service_id)
);

 * sqlite:///mario_bros_plumbing.db
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [13]:
import base64
from IPython.display import Image, display, HTML

def mm(graph):
    graphbytes = graph.encode("utf8")
    base64_bytes = base64.b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))


mm("""
classDiagram
    Customers "1" -- "*" Orders
    Employees "1" -- "*" Orders
    Orders "1" -- "*" Order_Items
    Services "1" -- "*" Order_Items
    ServiceTypes <|-- Services

    class Customers {
        +customer_id: INTEGER PK
        +first_name: TEXT
        +last_name: TEXT
        +address: TEXT
        +phone_number: TEXT
    }

    class Employees {
        +employee_id: INTEGER PK
        +first_name: TEXT
        +last_name: TEXT
        +job_title: TEXT
        +hire_date: TEXT
    }

    class ServiceTypes {
        +service_type_id: INTEGER PK
        +service_type_name: TEXT
        +description: TEXT
    }

    class Services {
        +service_id: INTEGER PK
        +service_type_id: INTEGER FK
        +service_name: TEXT
        +description: TEXT
        +price: REAL
    }

    class Orders {
        +order_id: INTEGER PK
        +customer_id: INTEGER FK
        +employee_id: INTEGER FK
        +order_date: TEXT
        +total_amount: REAL
    }

    class Order_Items {
        +order_item_id: INTEGER PK
        +order_id: INTEGER FK
        +service_id: INTEGER FK
        +quantity: INTEGER
    }
    """
  )

## Guide to Interpreting a UML Entity-Relationship Diagram
THe above diagram is in **Unified Modeling Language.** This is similar to the Crow's foot style we saw before, but with a few key differences.

### Entities:
  -   In UML, entities are represented as classes, which are depicted as rectangles.
  -   Each entity (class) is labeled with its name at the top of the rectangle.
  -   The attributes of an entity are listed inside the rectangle, below the entity name.
  -   In this diagram, the entities are Customers, Employees, ServiceTypes, Services, Orders, and Order_Items.-   Attributes:
  -   Attributes represent the properties or characteristics of an entity.
  -   Each attribute is listed on a separate line within the entity's rectangle.
  -   The attribute name is followed by a colon (`:`) and its data type.
  -   Primary key attributes are marked with `<<PK>>`, indicating that they uniquely identify each record in the entity.
  -   Foreign key attributes are marked with `<<FK>>`, indicating that they establish relationships with other entities.
  
### Relationships:
  -   Relationships between entities are represented by lines connecting the rectangles.
  -   The cardinality of a relationship is indicated at each end of the line.
  -   In this diagram, a single number (`1`) represents a one-to-one or one-to-many relationship, while an asterisk (`*`) represents a many-to-one or many-to-many relationship.
  -   For example, the line between Customers and Orders with `1` on the Customers end and `*` on the Orders end indicates that one customer can have multiple orders (a one-to-many relationship).

### Inheritance:
  -   Inheritance is a concept in object-oriented modeling where one entity (subclass) inherits the attributes and behaviors of another entity (superclass).
  -   In UML, inheritance is represented by a line with a hollow arrowhead pointing from the subclass to the superclass.
  -   In this diagram, the inheritance relationship is shown between Services and ServiceTypes, with Services inheriting from ServiceTypes.
    
### Reading the Diagram:
  -   Start by identifying the entities (classes) in the diagram, which are represented by rectangles.
  -   Look at the attributes within each entity to understand the properties and characteristics of each entity.
  -   Identify the primary key attributes marked with `<<PK>>` and foreign key attributes marked with `<<FK>>`.
  -   Follow the lines connecting the entities to understand the relationships between them.
  -   Look at the cardinality indicators (`1` or `*`) to determine the nature of the relationships (one-to-one, one-to-many, many-to-one, or many-to-many).
  -   Observe any inheritance relationships, where one entity inherits from another, indicated by a line with a hollow arrowhead.

In [3]:
%%sql
INSERT INTO Employees (employee_id, first_name, last_name, job_title, hire_date)
VALUES
  (1, 'Mario', 'Bro', 'Master Plumber', '1981-09-13'),
  (2, 'Luigi', 'Bro', 'Plumber', '1990-02-20'),
  (3, 'Peach', 'Toadstool', 'Manager', '2000-11-05'),
  (4, 'Birdo', 'Bird', 'Apprentice Plumber', '2015-03-10');

 * sqlite:///mario_bros_plumbing.db
4 rows affected.


[]

In [4]:
%%sql
INSERT INTO Customers (customer_id, first_name, last_name, address, phone_number)
VALUES
  (1, 'Peach', 'Toadstool', 'Mushroom Castle, Toad Town', '(555) 123-4567'),
  (2, 'Yoshi', 'Dino', '24 Egg Island, Dinosaur Land', '(555) 987-6543'),
  (3, 'Daisy', 'Sarasa', '10 Sarasaland Way, Chai Kingdom', '(555) 456-7890'),
  (4, 'Toadette', 'Toadstool', '15 Mushroom St, Toad Town', '(555) 789-0123'),
  (5, 'Bowser', 'Koopa', '1 Bowser Castle, Dark Land', '(555) 654-3210'),
  (6, 'Wario', 'Wario', '100 Gold Coin Blvd, Diamond City', '(555) 321-0987'),
  (7, 'Waluigi', 'Wario', '101 Silver Coin Ave, Diamond City', '(555) 098-7654'),
  (8, 'Donkey', 'Kong', '50 Banana Jungle, DK Island', '(555) 111-2222'),
  (9, 'Diddy', 'Kong', '51 Banana Jungle, DK Island', '(555) 333-4444'),
  (10, 'Rosalina', 'Cosmic', 'Comet Observatory, Space', '(555) 555-6666'),
  (11, 'Cappy', 'Bonneter', '1 Hat Kingdom Way, Cap Kingdom', '(555) 777-8888'),
  (12, 'Pauline', 'Toyopolis', '1 New Donk City Plaza, Metro Kingdom', '(555) 999-0000'),
   (13, 'Link', 'Hyrule', '1 Hyrule Castle, Hyrule Kingdom', '(555) 234-5678'),
  (14, 'Zelda', 'Hyrule', '2 Hyrule Castle, Hyrule Kingdom', '(555) 345-6789'),
  (15, 'Samus', 'Aran', '10 Bounty Hunter Ave, Space Colony K-2L', '(555) 456-7890'),
  (16, 'Fox', 'McCloud', '20 Corneria St, Lylat System', '(555) 567-8901'),
  (17, 'Pikachu', 'Pokemon', '30 Electric Rodent Rd, Kanto Region', '(555) 678-9012'),
  (18, 'Kirby', 'Puffball', '40 Dream Land Ln, Planet Popstar', '(555) 789-0123'),
  (19, 'Captain', 'Falcon', '50 Mute City Blvd, Port Town', '(555) 890-1234'),
  (20, 'Ness', 'Earthbound', '60 Onett Ave, Eagleland', '(555) 901-2345'),
  (21, 'Marth', 'Altea', '70 Fire Emblem St, Kingdom of Altea', '(555) 012-3456'),
  (22, 'Roy', 'Pherae', '71 Fire Emblem St, Lycia', '(555) 123-4567'),
  (23, 'Toad', 'Toadstool', '80 Mushroom Way, Toad Town', '(555) 234-5678'),
  (24, 'Koopa', 'Troopa', '90 Shell Blvd, Dark Land', '(555) 345-6789'),
  (25, 'Dry', 'Bones', '91 Bone Yard Rd, Dark Land', '(555) 456-7890'),
  (26, 'Shy', 'Guy', '100 Mask St, Dark Land', '(555) 567-8901'),
  (27, 'Lakitu', 'Cloud', '110 Cloud Way, Skyworld', '(555) 678-9012'),
  (28, 'Toadsworth', 'Toadstool', '120 Cane Ln, Toad Town', '(555) 789-0123'),
  (29, 'Birdo', 'Egg', '130 Egg Shooter Ave, Subcon', '(555) 890-1234'),
  (30, 'Nabbit', 'Thief', '140 Burglar Blvd, Acorn Plains', '(555) 901-2345'),
  (31, 'Funky', 'Kong', '150 Surf Shop St, DK Island', '(555) 012-3456'),
  (32, 'Cranky', 'Kong', '160 Rocking Chair Rd, DK Island', '(555) 123-4567');

 * sqlite:///mario_bros_plumbing.db
32 rows affected.


[]

In [5]:
%%sql
INSERT INTO ServiceTypes (service_type_id, service_type_name, description)
VALUES
  (1, 'Repair', 'Services related to fixing and repairing plumbing issues'),
  (2, 'Installation', 'Services related to installing new plumbing fixtures and systems'),
  (3, 'Inspection', 'Services related to inspecting and assessing plumbing systems');

INSERT INTO Services (service_id, service_type_id, service_name, description, price)
VALUES
  (1, 1, 'Pipe Repair', 'Fix leaky or broken pipes', 50.00),
  (2, 1, 'Drain Cleaning', 'Clear clogged drains and pipes', 75.00),
  (3, 2, 'Toilet Installation', 'Install a new toilet', 150.00),
  (4, 2, 'Sink Replacement', 'Replace an old or damaged sink', 200.00),
  (5, 1, 'Water Heater Repair', 'Fix issues with water heaters', 120.00),
  (6, 3, 'Plumbing Inspection', 'Inspect plumbing systems for potential issues', 100.00),
  (7, 1, 'Emergency Service', '24/7 emergency plumbing service', 250.00);

 * sqlite:///mario_bros_plumbing.db
3 rows affected.
7 rows affected.


[]

In [6]:
import sqlite3
import random
from datetime import datetime, timedelta

# Connect to the SQLite database
conn = sqlite3.connect('mario_bros_plumbing.db')
cursor = conn.cursor()

# Generate 500 random orders
for _ in range(500):
    # Select a random customer with weighted probabilities
    cursor.execute("SELECT customer_id FROM Customers ORDER BY RANDOM() * CASE "
                   "WHEN customer_id IN (1, 2, 3) THEN 3 "  # Customers 1, 2, and 3 are 3 times more likely
                   "WHEN customer_id IN (4, 5, 6) THEN 2 "  # Customers 4, 5, and 6 are 2 times more likely
                   "ELSE 1 END LIMIT 1")
    result = cursor.fetchone()
    if result is None:
        continue  # Skip this iteration if no customer is found
    customer_id = result[0]

    # Generate a random order date between 1985 and 2024
    start_date = datetime(1985, 1, 1)
    end_date = datetime(2024, 12, 31)
    time_between_dates = end_date - start_date
    days_between_dates = time_between_dates.days
    random_number_of_days = random.randrange(days_between_dates)
    order_date = start_date + timedelta(days=random_number_of_days)

    # Select a random employee hired before the order date
    employee_id = None  # Initialize employee_id with a default value
    cursor.execute("SELECT employee_id FROM Employees WHERE hire_date <= ? ORDER BY RANDOM() LIMIT 1",
                   (order_date.strftime('%Y-%m-%d'),))
    result = cursor.fetchone()
    if result is not None:
        employee_id = result[0]

    if employee_id is None:
        continue  # Skip this iteration if no suitable employee is found

    # Insert the order into the Orders table
    cursor.execute("INSERT INTO Orders (customer_id, employee_id, order_date) VALUES (?, ?, ?)",
                   (customer_id, employee_id, order_date.strftime('%Y-%m-%d')))
    order_id = cursor.lastrowid

    # Generate random order items for the order
    num_items = random.randint(1, 5)
    for _ in range(num_items):
        # Select a random service with weighted probabilities
        cursor.execute("SELECT service_id, price FROM Services ORDER BY RANDOM() * CASE "
                       "WHEN service_id IN (1, 2) THEN 3 "  # Services 1 and 2 are 3 times more likely
                       "WHEN service_id IN (3, 4) THEN 2 "  # Services 3 and 4 are 2 times more likely
                       "ELSE 1 END LIMIT 1")
        result = cursor.fetchone()
        if result is None:
            print("No service found for order", order_id)
            continue  # Skip this iteration if no service is found
        service_id, price = result

        # Generate a random quantity
        quantity = random.randint(1, 3)

        # Insert the order item into the Order_Items table
        cursor.execute("INSERT INTO Order_Items (order_id, service_id, quantity) VALUES (?, ?, ?)",
                       (order_id, service_id, quantity))

    # Update the total amount for the order
    cursor.execute("UPDATE Orders SET total_amount = (SELECT SUM(Services.price * Order_Items.quantity) "
                   "FROM Order_Items JOIN Services ON Order_Items.service_id = Services.service_id "
                   "WHERE Order_Items.order_id = ?) WHERE order_id = ?", (order_id, order_id))

# Commit the changes and close the connection
conn.commit()
conn.close()

In [7]:
%%sql
-- determine number of order per employee
SELECT employee_id, COUNT(*) AS num_orders
FROM Orders
GROUP BY employee_id
ORDER BY num_orders DESC;

 * sqlite:///mario_bros_plumbing.db
Done.


employee_id,num_orders
1,199
2,171
3,106
4,24


In [8]:
%%sql
SELECT * FROM Order_Items LIMIT 10;

 * sqlite:///mario_bros_plumbing.db
Done.


order_item_id,order_id,service_id,quantity
1,1,4,1
2,1,1,3
3,2,4,2
4,3,4,1
5,3,4,2
6,3,3,1
7,3,1,3
8,4,5,1
9,4,2,2
10,4,3,1


In [9]:
%%sql
-- Determine total amount spent by customer
SELECT Customers.first_name, Customers.last_name, SUM(total_amount) AS total_spent
FROM Orders
JOIN Customers ON Orders.customer_id = Customers.customer_id
GROUP BY Customers.customer_id
ORDER BY total_spent DESC;

 * sqlite:///mario_bros_plumbing.db
Done.


first_name,last_name,total_spent
Daisy,Sarasa,80295.0
Peach,Toadstool,67125.0
Yoshi,Dino,64920.0
Wario,Wario,37245.0
Toadette,Toadstool,32725.0
Bowser,Koopa,29220.0
Diddy,Kong,4600.0
Birdo,Egg,3100.0
Samus,Aran,2600.0
Lakitu,Cloud,1810.0
